### gpu 메모리 부족 현상이 발생해서 이에 대한 새로운 접근
mini_batch size: 24->12  
upsampling: (4,4) -> (2,2)

In [1]:
import tensorflow as tf
from tensorflow.keras import mixed_precision

# mixed precision 설정
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# GPU 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

2024-06-06 09:14:16.598047: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 09:14:16.721778: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPUs will likely run quickly with dtype policy mixed_float16 as they all have compute capability of at least 7.0


In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  4


# 데이터 불러오기 및 클래스 지정과 분할  
The numbers of samples in each class for the test set are distributed as 106 (normal), 24 (AOM), 13 (CSOM), 28 (Earwax), and 20 (Other)

In [3]:
import os

# 데이터셋의 루트 디렉토리
dataset_directory = '/home/jeonk636/ear_classification/eardrumDs'

# 클래스 이름 및 라벨을 매핑하기 위한 딕셔너리 생성
# normal:0, Aom:1, Chornic:2, Earwax:3, Others:4
label_map = {
    'Aom': 1,
    'Chornic': 2,
    'Earwax': 3,
    'Normal': 0,
}

# 이미지 파일 경로 및 라벨을 저장할 리스트
image_paths = []
image_labels = []

exclude_folders = {'OtitExterna', 'tympanoskleros', 'Earventulation', 'Foreign', 'PseduoMembran'}

# 각 폴더에 대해 이미지 파일 경로 및 해당 라벨을 리스트에 추가
for label_folder in os.listdir(dataset_directory):
    if label_folder in exclude_folders:  # 이 폴더는 건너뛴다.
        continue
    folder_path = os.path.join(dataset_directory, label_folder)
    
    for image_filename in os.listdir(folder_path):
        # 폴더 내 파일이 실제 파일인지 확인하고 '.ipynb_checkpoints' 폴더를 건너뛴다.
        full_path = os.path.join(folder_path, image_filename)
        if os.path.isfile(full_path) and '.ipynb_checkpoints' not in full_path:
            image_paths.append(full_path)
            image_labels.append(label_map[label_folder])


In [4]:
label_0_count = image_labels.count(0)
label_1_count = image_labels.count(1)
label_2_count = image_labels.count(2)
label_3_count = image_labels.count(3)

print('Normal:', label_0_count)
print('Aom:', label_1_count)
print('Chornic:', label_2_count)
print('Earwax:', label_3_count)

Normal: 534
Aom: 119
Chornic: 63
Earwax: 140


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

# 이미지 경로와 라벨을 Numpy 배열로 변환
image_paths = np.array(image_paths)
image_labels = np.array(image_labels)

# 데이터프레임으로 변환
data = pd.DataFrame({'image_path': image_paths, 'label': image_labels})

# 클래스별로 지정된 수의 샘플을 추출하여 테스트 세트로 사용(4:20 #other)
test_counts = {
    0: 106,  # Normal
    1: 24,   # AOM
    2: 13,   # CSOM
    3: 28   # Earwax
}

test_data_list = []

for label, count in test_counts.items():
    class_data = data[data['label'] == label]
    if len(class_data) < count:
        raise ValueError(f"Not enough samples for class {label}. Needed {count}, but only {len(class_data)} available.")
    test_data_list.append(class_data.sample(n=count, random_state=42))

test_data = pd.concat(test_data_list)

# 나머지 데이터를 훈련 세트로 사용
train_data = data.drop(test_data.index)

# 훈련 세트와 테스트 세트로 나누기
x_train = train_data['image_path'].values
y_train = train_data['label'].values
x_test = test_data['image_path'].values
y_test = test_data['label'].values

# # train 8: validation 2 분할
# x_train, x_test, y_train, y_test = train_test_split(
#     image_paths, image_labels, stratify=image_labels, test_size=0.2, random_state=42)

In [6]:
# print("Train set:", len(x_train))
# print("test set:", len(x_test))

In [7]:
# 결과 출력
print("Training set size:", len(x_train))
print("Test set size:", len(x_test))
print("Test set class distribution:\n", test_data['label'].value_counts())

Training set size: 685
Test set size: 171
Test set class distribution:
 label
0    106
3     28
1     24
2     13
Name: count, dtype: int64


# keras을 사용한 데이터 증강  
224 by 224  
Normal 클래스가 가장 많은 샘플 수를 가지고 있으므로, 이를 기준으로 AOM, CSOM, Earwax 클래스을 각각 3배, 7배, 3배 증강  
증강을 하면 train set에 있는 total sample 수는 normal 428, Aom 380, Csom 400, Earwax 448 으로 각 클래스가 비슷해짐

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 레이블을 문자열로 변환
train_data['label'] = train_data['label'].astype(str)
test_data['label'] = test_data['label'].astype(str)

# 데이터 증강 파라미터 정의
data_gen_args = dict(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.7, 1.3],  # ±30% 밝기 조절
    width_shift_range=0.2,  # 좌우 이동 범위 20%
    height_shift_range=0.2,  # 상하 이동 범위 20%
    zoom_range=0.2,  # ±20% 확대/축소
    rotation_range=20  # ±20도 회전
)

# 데이터 증강 제네레이터 생성
datagen = ImageDataGenerator(**data_gen_args)

# 증강된 샘플 수를 지정(4:4 #Other)
augment_counts = {
    1: 3,  # AOM
    2: 7,  # CSOM
    3: 3  # Earwax
}

# 증강된 데이터를 저장할 리스트
augmented_data = []

for label, count in augment_counts.items():
    class_data = train_data[train_data['label'] == str(label)]
    for i in range(count):
        for index, row in class_data.iterrows():
            img = tf.keras.preprocessing.image.load_img(row['image_path'], target_size=(224, 224))
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            augmented_iter = datagen.flow(img_array, batch_size=1)
            augmented_img = next(augmented_iter)[0].astype(np.uint8)
            augmented_data.append({'image_path': row['image_path'], 'label': row['label']})
            
# 원본 데이터와 증강 데이터를 결합
augmented_df = pd.DataFrame(augmented_data)
train_data = pd.concat([train_data, augmented_df])

In [9]:
# 데이터 확인
print(train_data['label'].value_counts())
print(augmented_df['label'].value_counts())

label
3    448
0    428
2    400
1    380
Name: count, dtype: int64
label
2    350
3    336
1    285
Name: count, dtype: int64


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, Activation, Input, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# logical_gpus = tf.config.experimental.list_logical_devices ('GPU')
# strategy = tf.distribute.MirroredStrategy(logical_gpus)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # 데이터 증강 설정
    data_gen_args = dict(
        rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True,
        brightness_range=[0.7, 1.3],
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2
    )

    # 데이터 증강 생성기
    train_datagen = ImageDataGenerator(**data_gen_args)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # 데이터 증강 적용 (train_data는 데이터프레임 형태라고 가정)
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_data,
        x_col='image_path',
        y_col='label',
        target_size=(224, 224),
        batch_size=4,  # 배치 크기를 줄임
        class_mode='categorical'
    )

    val_generator = test_datagen.flow_from_dataframe(
        dataframe=test_data,
        x_col='image_path',
        y_col='label',
        target_size=(224, 224),
        batch_size=4,  # 배치 크기를 줄임
        class_mode='categorical'
    )

    # Convolutional Block 정의
    def conv_block(x, filters):
        x = Conv2D(filters, (3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        return x

    # Residual Block 정의
    def residual_block(x, filters):
        shortcut = x
        x = Conv2D(filters, (3, 3), padding='same', strides=(1, 1))(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Conv2D(filters, (3, 3), padding='same', strides=(1, 1))(x)
        x = BatchNormalization()(x)
        x = tf.keras.layers.add([x, shortcut])
        x = LeakyReLU()(x)
        return x

    # Channel Attention Module 정의
    def channel_attention(x):
        avg_pool = GlobalAveragePooling2D()(x)
        max_pool = GlobalAveragePooling2D()(x)
        dense_avg = Dense(x.shape[-1], activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')(avg_pool)
        dense_max = Dense(x.shape[-1], activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')(max_pool)
        combined = tf.keras.layers.add([dense_avg, dense_max])
        channel_attention = Activation('sigmoid')(combined)
        return tf.keras.layers.multiply([x, channel_attention])

    # Spatial Attention Module 정의
    def spatial_attention(x):
        avg_pool = tf.reduce_mean(x, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(x, axis=-1, keepdims=True)
        combined = Concatenate(axis=-1)([avg_pool, max_pool])
        spatial_attention = Conv2D(1, (7, 7), padding='same', activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(combined)
        return tf.keras.layers.multiply([x, spatial_attention])

    # CBAM Block 정의
    def cbam_block(x):
        x = channel_attention(x)
        x = spatial_attention(x)
        return x

    # Stage 1 정의
    def stage1(input_tensor):
        x = conv_block(input_tensor, 32)
        x = conv_block(x, 32)
        x = MaxPooling2D((2, 2))(x)
        x = UpSampling2D((4, 4), interpolation='bilinear')(x)  
        return x

    # Stage 2 정의
    def stage2(input_tensor):
        x = conv_block(input_tensor, 64)
        x = cbam_block(x)
        x = residual_block(x, 64)
        x = MaxPooling2D((2, 2))(x)
        x = UpSampling2D((4, 4), interpolation='bilinear')(x)  
        return x

    # Stage 3 정의
    def stage3(input_tensor):
        x = conv_block(input_tensor, 128)
        x = cbam_block(x)
        x = residual_block(x, 128)
        x = MaxPooling2D((2, 2))(x)
        x = UpSampling2D((4, 4), interpolation='bilinear')(x)  
        return x

    # 모델 구성
    input_tensor = Input(shape=(224, 224, 3))

    stage1_output = stage1(input_tensor)
    stage2_output = stage2(stage1_output)
    stage3_output = stage3(stage2_output)

    #concatenated = Concatenate()([stage1_output, stage2_output, stage3_output])

    x = GlobalAveragePooling2D()(stage3_output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    output = Dense(4, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output)

    # 모델 컴파일
    model.compile(optimizer=Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999), loss='categorical_crossentropy', metrics=['accuracy'])

    # 모델 학습
    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=100,
        validation_data=val_generator,
        validation_steps=len(val_generator)
    )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Found 1656 validated image filenames belonging to 4 classes.
Found 171 validated image filenames belonging to 4 classes.


2024-06-06 09:19:04.787612: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\020TensorDataset:40"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_INT32
        }
      }
    }
  }
}



Epoch 1/100
INFO:tensorflow:batch_all_reduce: 46 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 46 all-reduces with algorithm = nccl, num_packs = 1


2024-06-06 09:19:36.895387: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fe4f0004ac0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-06 09:19:36.895458: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2024-06-06 09:19:36.895473: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): Quadro RTX 5000, Compute Capability 7.5
2024-06-06 09:19:36.895484: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (2): Quadro RTX 5000, Compute Capability 7.5
2024-06-06 09:19:36.895494: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (3): Quadro RTX 5000, Compute Capability 7.5
2024-06-06 09:19:36.907005: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-06 09:19:36.994529: I tens

414/414 [==============================] - ETA: 0s - loss: 1.4033 - accuracy: 0.2609

2024-06-06 09:21:39.609932: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\020TensorDataset:80"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_INT32
        }
      }
    }
  }
}



414/414 [==============================] - 160s 303ms/step - loss: 1.4033 - accuracy: 0.2609 - val_loss: 1.3833 - val_accuracy: 0.1930
Epoch 2/100
414/414 [==============================] - 123s 298ms/step - loss: 1.3953 - accuracy: 0.2687 - val_loss: 1.2788 - val_accuracy: 0.4912
Epoch 3/100
414/414 [==============================] - 124s 298ms/step - loss: 1.3988 - accuracy: 0.2421 - val_loss: 1.2560 - val_accuracy: 0.4854
Epoch 4/100
414/414 [==============================] - 124s 299ms/step - loss: 1.3857 - accuracy: 0.2766 - val_loss: 1.2349 - val_accuracy: 0.5088
Epoch 5/100
414/414 [==============================] - 124s 298ms/step - loss: 1.3829 - accuracy: 0.2699 - val_loss: 1.2410 - val_accuracy: 0.4795
Epoch 6/100
414/414 [==============================] - 124s 298ms/step - loss: 1.3826 - accuracy: 0.2886 - val_loss: 1.2639 - val_accuracy: 0.5146
Epoch 7/100
414/414 [==============================] - 124s 298ms/step - loss: 1.3834 - accuracy: 0.2844 - val_loss: 1.2728 - val_

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# 학습 결과 시각화
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(100)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Stage 1 summary
stage1_output = stage1(input_tensor)
stage1_model = Model(inputs=input_tensor, outputs=stage1_output)
print("Stage 1 Summary:")
stage1_model.summary()

In [ ]:
# Stage 2 summary
stage2_output = stage2(stage1_output)
stage2_model = Model(inputs=input_tensor, outputs=stage2_output)
print("\nStage 2 Summary:")
stage2_model.summary()

In [ ]:
# Stage 3 summary
stage3_output = stage3(stage2_output)
stage3_model = Model(inputs=input_tensor, outputs=stage3_output)
print("\nStage 3 Summary:")
stage3_model.summary()